In [10]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

csv_files = glob.glob('./csv_ave/*.csv')  # Assuming CSVs are stored in 'data/' folder

X, Y = [], []  # Lists to store input and output data
df_all = pd.read_csv("all_c6.csv")
n = 0

for file in csv_files:
    df = pd.read_csv(file)  # Load temperature data file
    
    # Extract temperature sequence (all rows)
    avg_temp_sequence = df["ave"] # (time_steps,)


    Y.append(avg_temp_sequence.size)  # Shape: (time_steps,)


print(Y)

[43664, 42816, 22098, 10709, 28965, 35368, 34455, 15497, 10666, 15648, 22232, 28338]


In [9]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

csv_files = glob.glob('./csv_ave/*.csv')  # Assuming CSVs are stored in 'data/' folder

X, Y = [], []  # Lists to store input and output data
df_all = pd.read_csv("all_c6.csv")
n = 0

for file in csv_files:
    df = pd.read_csv(file)  # Load temperature data file
    
    # Extract metadata from df_all
    initial_speed = df_all["v_set\n[km/h]"][n]
    initial_pressure = df_all["Fb_set\n[kN]"][n]
    
    # Extract temperature sequence (all rows)
    avg_temp_sequence = df["ave"] # (time_steps,)

    # Normalize inputs
    initial_speed /= 100  # Assuming max speed ~100 km/h
    initial_pressure /= 23  # Assuming max pressure ~23 kN

    # Store in lists
    X.append([initial_speed, initial_pressure])  # Shape: (2,)
    Y.append(avg_temp_sequence)  # Shape: (time_steps,)

    n = n+ 1
    #print(n)

# Convert to numpy arrays
X = np.array(X)  # Shape: (num_tests, 2)
Y = np.array(Y)  # Shape: (num_tests, time_steps)

# Reshape X for LSTM (samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))  # Shape: (num_tests, 1, 2)

# Reshape Y for LSTM (samples, timesteps, features)
Y = Y.reshape((Y.shape[0], Y.shape[1], 1))  # Shape: (num_tests, time_steps, 1)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (12,) + inhomogeneous part.

In [2]:
print(n)

0


In [ ]:
# Define LSTM model
model = Sequential([
    Dense(16, activation="relu", input_shape=(1, 2)),  # Input: speed, pressure
    RepeatVector(Y.shape[1]),  # Repeat input vector for each time step
    LSTM(64, return_sequences=True),  # LSTM learns temperature pattern
    TimeDistributed(Dense(32, activation="relu")),  # Dense applied at each time step
    TimeDistributed(Dense(1))  # Output: temperature sequence
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model
model.fit(X, Y, epochs=20, batch_size=4, validation_split=0.2)

# Save model for future predictions
model.save("brake_temp_lstm.h5")


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load trained model
model = load_model("brake_temp_lstm.h5")

# Example input (Speed = 120 km/h, Pressure = 60 bar)
new_input = np.array([[120 / 200, 60 / 100]])  # Normalize values
new_input = new_input.reshape((1, 1, 2))  # Reshape for LSTM

# Predict temperature sequence
predicted_temp_sequence = model.predict(new_input)

# Print results
print(predicted_temp_sequence[0].flatten())  # Output: Temperature over time
